In [ ]:
!pip install scikit-learn==1.5.2
# run this line without the '!' in your terminal to have it installed locally
# then you can skip running this



# create days_since_april_1 column
# convert dateTime to days out of 365
# for each value, add 365, subtract april 1st as a number of days
# for each value, mod by 365


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [15]:
# Load dataset
housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
y = np.log1p(housing["price"])  # Log transform target

X = housing.drop(["price", "id", "date"], axis=1)
X = pd.get_dummies(X)

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       20000 non-null  int64  
 1   bathrooms      20000 non-null  float64
 2   sqft_living    20000 non-null  int64  
 3   sqft_lot       20000 non-null  int64  
 4   floors         20000 non-null  float64
 5   waterfront     20000 non-null  int64  
 6   view           20000 non-null  int64  
 7   condition      20000 non-null  int64  
 8   grade          20000 non-null  int64  
 9   sqft_above     20000 non-null  int64  
 10  sqft_basement  20000 non-null  int64  
 11  yr_built       20000 non-null  int64  
 12  yr_renovated   20000 non-null  int64  
 13  zipcode        20000 non-null  int64  
 14  lat            20000 non-null  float64
 15  long           20000 non-null  float64
 16  sqft_living15  20000 non-null  int64  
 17  sqft_lot15     20000 non-null  int64  
dtypes: flo

In [5]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create XGBoost regressor
model = xgb.XGBRegressor(objective="reg:squarederror", eval_metric="rmse", subsample=0.6,
                         n_estimators=1500, max_depth=7, learning_rate=0.01, colsample_bytree=0.7)

# Train model
model.fit(X_train, y_train)

# Make predictions on test set
y_pred = model.predict(X_test)

In [ ]:
# Evaluate performance
y_non_log_test = np.expm1(y_test)
y_non_log_pred = np.expm1(y_pred)

mse = mean_squared_error(y_non_log_test, y_non_log_pred)
rmse = np.sqrt(mse)  # Compute RMSE
# return from a log of cost to cost
# rmse = np.expm1(rmse)
r2 = r2_score(y_test, y_pred)  # Compute R^2
r2_2 = r2_score(y_non_log_test, y_non_log_pred)  # Compute R^2

print(f"R² Score: {r2:.4f} {r2_2:.4f}")
# print(f"RMSE: {rmse:.4f}")
y_non_log_test

R² Score: 0.9058 0.8853


C:\Users\david\AppData\Local\Temp\ipykernel_10216\894392122.py:8: RuntimeWarning: overflow encountered in expm1
  rmse = np.expm1(rmse)


10650    825000.0
2041     532500.0
8668     435000.0
1114     216000.0
13902    525000.0
           ...   
4073     275000.0
7442     272000.0
9999     328423.0
1870     565000.0
15196    437000.0
Name: price, Length: 4000, dtype: float64

In [12]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

ValueError: continuous is not supported

In [8]:
# Load the holdout dataset
holdout_data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv')

# Drop unnecessary columns
X_holdout = holdout_data.drop(["id", "date"], axis=1)

# Predict using the trained model
y_holdout_pred = model.predict(X_holdout)

# Convert predictions back to original scale
y_holdout_pred_original = np.expm1(y_holdout_pred)

# Create DataFrame with correct format
predictions_df = pd.DataFrame({"price": y_holdout_pred_original})


In [9]:

# Save in the required format
team_name = "team8"  # Replace with your actual team name
filename = f"{team_name}-module3-predictions.csv"
predictions_df.to_csv(filename, index=False)

print(f"Predictions saved as {filename}")

Predictions saved as team8-module3-predictions.csv
